In [1]:
# from theano.sandbox import cuda

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras import optimizers
from keras.models import Sequential

Using Theano backend.


In [2]:
model_path = '/Users/mdymshits/fastai/data/imdb/models/'

In [ ]:
# %mkdir -p /Users/mdymshits/fastai/data/imdb/models/

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [3]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [4]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [5]:
idx2word = {v: k for k, v in idx.iteritems()}

We download the reviews using code copied from keras.datasets:

In [6]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [7]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [10]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [11]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

Reduce vocab size by setting rare words to max index.

In [8]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [9]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [10]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [16]:
trn.shape

(25000, 500)

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [11]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [12]:
%ls -1 /Users/mdymshits/fastai/data/glove/results/

6B.50d.dat/
6B.50d.tgz
6B.50d_idx.pkl
6B.50d_words.pkl
glove.6B.100d.txt
glove.6B.200d.txt
glove.6B.300d.txt
glove.6B.50d.txt
glove.6B.zip
glove.twitter.27B.100d.txt
glove.twitter.27B.200d.txt
glove.twitter.27B.25d.txt
glove.twitter.27B.50d.txt
glove.twitter.27B.zip


In [13]:
vecs, words, wordidx = load_vectors('/Users/mdymshits/fastai/data/glove/results/6B.50d')

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [14]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))
    counter = 0
    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            counter += 1
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb, counter

In [15]:
emb, c = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [16]:
emb.shape

(5000, 50)

In [18]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])
#     Dense(2, activation ='softmax')])

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

**previous results**:
```
Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 11s - loss: 0.3997 - acc: 0.8207 - val_loss: 0.3032 - val_acc: 0.8943
Epoch 2/2
25000/25000 [==============================] - 11s - loss: 0.2882 - acc: 0.8832 - val_loss: 0.2646 - val_acc: 0.9029
```

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 500, 50)       250000      embedding_input_2[0][0]          
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 500, 50)       0           embedding_2[0][0]                
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 500, 64)       16064       dropout_4[0][0]                  
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 500, 64)       0           convolution1d_2[0][0]            
___________________________________________________________________________________________

In [19]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 156s - loss: 0.5896 - acc: 0.6755 - val_loss: 0.5652 - val_acc: 0.6929
Epoch 2/5
25000/25000 [==============================] - 156s - loss: 0.4994 - acc: 0.7603 - val_loss: 0.4452 - val_acc: 0.8105
Epoch 3/5
25000/25000 [==============================] - 1538s - loss: 0.4803 - acc: 0.7726 - val_loss: 0.4687 - val_acc: 0.7788
Epoch 4/5
25000/25000 [==============================] - 167s - loss: 0.4627 - acc: 0.7832 - val_loss: 0.4242 - val_acc: 0.8257
Epoch 5/5
25000/25000 [==============================] - 170s - loss: 0.4500 - acc: 0.7893 - val_loss: 0.4246 - val_acc: 0.8149


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [21]:
model.layers[0].trainable=True

In [22]:
model.optimizer.lr=1e-4

In [23]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 4377s - loss: 0.4394 - acc: 0.7951 - val_loss: 0.4168 - val_acc: 0.8191


As expected, that's given us a nice little boost. :)

In [ ]:
model.save_weights(model_path+'glove50.h5')

In [ ]:
model.layers[0].trainable=False

In [ ]:
model.optimizer.lr=1e-5

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

This more complex architecture has given us another boost in accuracy.

## LSTM

We haven't covered this bit yet!

In [ ]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
              W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100, consume_less='gpu'),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)